In [2]:


# Build path từ root
IMAGE_DIR = "../content/cvpr2016_flowers/images"
CAPTION_DIR = "../content/cvpr2016_flowers/captions"

from load_captions import load_captions

captions = load_captions(IMAGE_DIR, CAPTION_DIR)
list(captions.items())[:5]

[('image_07088', 'dark blue little petals white green and yellow leaves'),
 ('image_06396',
  'this flower is blue and white in color, with petals that are wavy and ruffled.'),
 ('image_05847', 'the petals of this flower are pink with a short stigma'),
 ('image_04581',
  'a purple and red flower with rolled petals and light green sepal.'),
 ('image_03588', 'the petals of this flower are pink with a long stigma')]

In [3]:
from encode_captions import encode_captions

encoded_caption = encode_captions(captions)
print(encoded_caption["image_00001"]["embed"].shape)



/Users/nguyenhuynh/Documents/text-2-image/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([768])


In [4]:
from transform import transform
from preprocessing import FlowerDataset

dataset = FlowerDataset(IMAGE_DIR, captions=encoded_caption, transform=transform)
print(dataset[0]["image"].shape)
print(dataset[0]["text"])

torch.Size([3, 128, 128])
dark blue little petals white green and yellow leaves


In [6]:
import torch
from model import Generator

generator = Generator(noise_size=100, feature_size=128, num_channels=3, embedding_size=768, reduced_dim_size=256)
#test
batch_size = 5
# [batch_size, noise_size]
noise = torch.randn(size=(batch_size, 100))
print(noise.shape)
# [batch_size, embedding_size]
embedding_caption = dataset[0]["embed"].unsqueeze(0).repeat(batch_size, 1)
print(embedding_caption.shape)
output = generator(noise, embedding_caption)
output.shape


torch.Size([5, 100])
torch.Size([5, 768])


torch.Size([5, 3, 64, 64])

In [17]:
from model import Discriminator

images = torch.rand(size=(batch_size, 3, 128, 128))
discriminator = Discriminator(3, 128, 768, 256)
image = dataset[0]["image"].unsqueeze(0).repeat(batch_size, 1, 1, 1)

output, _ = discriminator(image, embedding_caption)
output.shape

torch.Size([5, 1])